# Azure AI Search と NVIDIA NIM および LlamaIndex の統合

このノートブックでは、NVIDIAのAIモデルとLlamaIndexを活用して強力なRetrieval-Augmented Generation (RAG)パイプラインを構築する方法を示します。NVIDIAのLLMと埋め込みを使用し、それらをAzure AI Searchのベクトルストアに統合し、RAGを実行して検索の品質と効率を向上させます。

## 利点
- **スケーラビリティ**: NVIDIAの大規模言語モデルとAzure AI Searchを活用して、スケーラブルで効率的な検索を実現。
- **コスト効率**: 効率的なベクトルストレージとハイブリッド検索技術で検索と取得を最適化。
- **高性能**: 強力なLLMとベクトル化された検索を組み合わせて、より迅速で正確な応答を提供。
- **品質**: 関連する取得文書を基にLLMの応答を補強し、高品質な検索を維持。

## 前提条件
- 🐍 Python 3.9以上
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA APIキー (NVIDIA NIMマイクロサービスを通じてNVIDIAのLLMと埋め込みにアクセスするため)

## 対応する機能
- ✅ NVIDIA LLMの統合 (使用するモデル: [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA埋め込み (使用するモデル: [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Azure AI Searchの高度な取得モード
- ✅ LlamaIndexを使用した文書インデックス化
- ✅ NVIDIA LLMを活用したAzure AI SearchとLlamaIndexによるRAG

さあ、始めましょう！


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## インストールと必要条件
Pythonバージョン >3.10 を使用してPython環境を作成してください。

## はじめに！


NVIDIA AI Foundationモデルを使用するには、`NVIDIA_API_KEY`が必要です：
1) [NVIDIA](https://build.nvidia.com/explore/discover)で無料アカウントを作成します。
2) 使用したいモデルをクリックします。
3) 入力欄でPythonタブを選択し、**Get API Key**をクリックしてから**Generate Key**をクリックします。
4) 生成されたキーをコピーしてNVIDIA_API_KEYとして保存します。これでエンドポイントへのアクセスが可能になります。


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAGの例：LLMと埋め込みの活用
### 1) LLMの初期化
`llama-index-llms-nvidia`、別名NVIDIAのLLMコネクタは、NVIDIA APIカタログで利用可能な互換性のあるモデルに接続し、生成することを可能にします。チャット補完モデルの一覧はこちらをご覧ください: https://build.nvidia.com/search?term=Text-to-Text

ここでは、**mixtral-8x7b-instruct-v0.1**を使用します。


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) 埋め込みの初期化
`llama-index-embeddings-nvidia`、別名NVIDIAのEmbeddingsコネクタは、NVIDIA APIカタログで利用可能な互換性のあるモデルに接続し、それらから生成することを可能にします。埋め込みモデルとして`nvidia/nv-embedqa-e5-v5`を選択しました。テキスト埋め込みモデルの一覧はこちらをご覧ください: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Azure AI Search ベクターストアを作成する


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) データに関する質問をするためのクエリエンジンを作成する

以下は、Azure AI Searchで純粋なベクター検索を使用し、応答を私たちのLLM（Phi-3.5-MOE）に基づかせたクエリの例です。


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Azure AI Searchでハイブリッド検索を使用したクエリです。


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### ベクター検索分析
LLMの回答は、ロシアの株式市場に関する経済的影響について、元のテキストで述べられている重要な点を正確に捉えています。具体的には、ロシアの株式市場が大幅に下落し、価値の40%を失ったこと、そして現在の状況により取引が停止されたことを述べています。この回答は元の情報とよく一致しており、LLMがロシアの行動や課された制裁による株式市場への影響について、関連する詳細を正しく特定し要約したことを示しています。

#### ソースノードのコメント
ソースノードは、国際的な制裁によってロシアが直面した経済的影響について詳しく説明しています。テキストでは、ロシアの株式市場が価値の40%を失い、取引が停止されたことが強調されています。また、ルーブルの価値の下落やロシア経済の広範な孤立といったその他の経済的影響についても言及されています。LLMの回答は、これらのノードから重要なポイントを効果的に抽出し、クエリで求められた株式市場への影響に焦点を当てています。


では、ハイブリッド検索が十分な根拠のある回答を提供できないケースを見てみましょう:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### ハイブリッド検索: LLM応答分析
ハイブリッド検索の例におけるLLMの応答は、提供されたコンテキストがロシアのウクライナ侵攻の正確な日付を特定していないことを示しています。この応答は、LLMがソース文書に含まれる情報を活用しつつ、テキスト内に具体的な詳細が欠けていることを認識していることを示唆しています。

この応答は、コンテキストがロシアの侵略に関連する出来事を言及しているものの、具体的な侵攻日を特定していないことを正確に指摘しています。これは、提供された情報を理解しながら、内容の欠落を認識するLLMの能力を示しています。LLMは、情報が不完全な場合に慎重さを示し、ユーザーに外部の情報源や歴史的記録を探すよう促す効果的な働きをしています。

### ソースノード分析
ハイブリッド検索の例におけるソースノードには、米国がロシアのウクライナでの行動に対応するための演説の抜粋が含まれています。これらのノードは、侵攻に対する米国およびその同盟国の対応と、より広範な地政学的影響を強調していますが、具体的な侵攻日については言及していません。これは、コンテキストに正確な日付情報が欠けていることを正しく指摘したLLMの応答と一致しています。


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### ハイブリッド w/リランキング: LLM応答分析
ハイブリッド w/リランキングの例では、LLMの応答が、イベントがスピーチの6日前に発生したことを指摘することで追加の文脈を提供しています。これは、LLMがスピーチのタイミングに基づいて侵攻の日付を推測できることを示していますが、正確さを求めるにはスピーチの日付を正確に知る必要があることも示しています。

この応答は、文脈の手がかりを活用してより情報量の多い答えを提供する能力が向上していることを示しています。また、リランキングの利点を強調しており、LLMがより関連性の高い情報にアクセスし、それを優先することで、求められる詳細（例:侵攻の日付）に近い答えを提供できることを示しています。

### ソースノード分析
この例のソースノードには、ロシアの侵攻のタイミングに関する参照が含まれており、具体的にはスピーチの6日前に発生したことが言及されています。正確な日付は明示されていないものの、ノードは時間的な文脈を提供しており、LLMがより微妙な応答をすることを可能にしています。この詳細の追加は、リランキングが提供された文脈から情報を抽出し推測するLLMの能力を向上させ、より正確で情報量の多い応答を生み出す方法を示しています。


**注意:**
このノートブックでは、NVIDIA APIカタログから提供されるNVIDIA NIMマイクロサービスを使用しました。  
上記のAPI、`NVIDIA (llms)`、`NVIDIAEmbedding`、および[Azure AI Search Semantic Hybrid Retrieval (組み込みの再ランキング機能)](https://learn.microsoft.com/azure/search/semantic-search-overview)を使用しています。  
なお、これらのAPIはセルフホスト型のマイクロサービスにも対応しています。

**例:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は一切の責任を負いません。
